# Column generation CVRP

In [1]:
import os
dir_notebooks = os.getcwd()
dir_home, _ = os.path.split(dir_notebooks)

dir_data = os.path.join(dir_home, 'data')
dir_code = os.path.join(dir_home, 'cvrp')

dir_instances = os.path.join(dir_data, 'instances')
dir_solutions = os.path.join(dir_data, 'solutions')
dir_my_solutions = os.path.join(dir_data, 'my_solutions')

import vrplib # https://github.com/leonlan/VRPLIB
import networkx as nx  
import numpy as np

# Import my modules
import sys
sys.path.append(f"{dir_home}") 
sys.path.append(f"{dir_code}")

from cvrp.mp_cvrp import MP_CVRP #Implementar CVRP
from utils.utils import save_solution
import pylgrim 

In [2]:
# Read VRPLIB formatted instances
instance = vrplib.read_instance(f"{dir_instances}/A-n32-k5.vrp")
solution = vrplib.read_solution(f"{dir_solutions}/A-n32-k5.sol") # only 1 solution format

In [3]:
instance["node_coord"] = instance["node_coord"][:10]
instance["capacity"] = 30

In [4]:
instance.keys()

dict_keys(['name', 'comment', 'type', 'dimension', 'edge_weight_type', 'capacity', 'node_coord', 'demand', 'depot', 'edge_weight'])

In [4]:
name = instance["name"]
n_nodes = len(instance["node_coord"])

nodes = list(range(n_nodes))
customers = nodes[1:] # 0 is the depot
depot = 0
n_vehicles = 5 # Por la instancia

arcs = [(i,j) for i in nodes for j in nodes if i!=j]

In [5]:
mp_cvrp = MP_CVRP(name, customers, n_vehicles)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-10


In [6]:
mp_cvrp.solve(time_limit = 60*60)
dual_customers, dual_depot = mp_cvrp.get_duals()

In [7]:
# Process duals
reduced_costs = {(i,j) : 0 for i,j in arcs}
for i,j in arcs:
    if j != depot:
        reduced_costs[i,j] = instance["edge_weight"][i,j] - dual_customers.get(j,0)
    else:
        reduced_costs[i,j] = instance["edge_weight"][i,j] - dual_depot

ESSPRC

In [9]:
# Create graph
G = nx.DiGraph(n_res=1) # Capacity
for i,j in arcs:
    G.add_edge(i,j, weight=reduced_costs[i,j], res_cost=np.array([instance["demand"][j]]))

In [10]:
source = 0
print('Testing with {} nodes'.format(len(G)))

Testing with 10 nodes


In [11]:
from pylgrim import tools
source_in = 'source_in'
tools.decouple_source(G, source, source_in=source_in)

9

In [12]:
from pylgrim import ESPPRC
target = source_in
max_res = list([instance["capacity"]])
G_pre, res_min = ESPPRC.preprocess(G, source, target, max_res, res_name='res_cost')
shortest_path, shortest_path_label = ESPPRC.GSSA(G_pre, source, target, max_res, res_min, res_name='res_cost')


print('shortest path found: {} with label {}'.format(shortest_path, shortest_path_label))
print('')

while True:
    try:
        e = shortest_path.__next__()
        print('{} ⇨ {} : {}'.format(*e))
        print('')
    except StopIteration:
        # last element reached
        break

shortest path found: 0 ⇨ 6 ⇨ 3 ⇨ 8 ⇨ source_in with label (-29800.25828322178, array([24.,  1.,  1.,  1.,  1.]))

0 ⇨ 6 : {'weight': -9948.115512915709, 'res_cost': array([12])}

6 ⇨ 3 : {'weight': -9976.230271351991, 'res_cost': array([6])}

3 ⇨ 8 : {'weight': -9961.516237190212, 'res_cost': array([6])}

8 ⇨ source_in : {'weight': 85.60373823613078, 'res_cost': array([0])}



In [ ]:
## La heuristica es muy muy lenta. Hay que pasarse a C++